In [ ]:
!pip install ipython-sql

# The code install the ipython-sql package using the pip package manager.
# pip is a tool used to install and manage Python packages. 
# The ! symbol at the beginning of the line is used in Jupyter notebooks to run shell commands.

In [2]:
import pandas as pd
import sqlite3

In [3]:
%load_ext sql

# This is a Jupyter notebook magic command that loads an SQL extension.
# This allows user to execute SQL queries directly in the Jupyter notebook.
# We can connect to any database which is supported by SQLAlchemy, in this project we will use SQLite database.

In [4]:
%sql sqlite:///MovieNow.db
# %sql is the magic command to connect to a SQLite database.
# The command sets the connection string to the default SQLite database.

##### Creating a database

In [ ]:
# Create blank tables inside SQLite database
%sql --file ./MovieNow-sqlite.sql

In [67]:
# Load Actors CSV file into a DataFrame
actors = 'actors.csv'
actsin = 'actsin.csv'
customers = 'customers.csv'
movies = 'movies.csv'
renting = 'renting.csv'

df_at = pd.read_csv(actors)
df_as = pd.read_csv(actsin)
df_c = pd.read_csv(customers)
df_m = pd.read_csv(movies)
df_r = pd.read_csv(renting)

# Connect to SQLite database
conn = sqlite3.connect('MovieNow.db')

# Write DataFrame to SQLite table
df_at.to_sql('actors', conn, index=False, if_exists='replace')
df_as.to_sql('actsin', conn, index=False, if_exists='replace')
df_c.to_sql('customers', conn, index=False, if_exists='replace')
df_m.to_sql('movies', conn, index=False, if_exists='replace')
df_r.to_sql('renting', conn, index=False, if_exists='replace')

# Close the connection
conn.close()

##### Showing the tables

In [5]:
%%sql
SELECT *
FROM actors
LIMIT 5;

 * sqlite:///MovieNow.db
Done.


actor_id,name,year_of_birth,nationality,gender
1,Abbie Cornish,1982.0,Australia,female
2,Adam Sandler,1966.0,USA,male
3,Al Pacino,1940.0,USA,male
4,Amy Adams,1974.0,USA,female
5,Andrea Riseborough,1981.0,British,female


In [7]:
%%sql
SELECT *
FROM actsin
LIMIT 5;

 * sqlite:///MovieNow.db
Done.


actsin_id,movie_id,actor_id
1,37,1
2,56,2
3,10,3
4,14,3
5,29,3


In [10]:
%%sql
SELECT *
FROM customers
LIMIT 5;

 * sqlite:///MovieNow.db
Done.


id,name,nationality,gender,dob,membersince
1,Robert Bohm,Austria,male,1980-07-30,2018-09-02
2,Wolfgang Ackermann,Austria,male,1971-11-17,2018-10-15
3,Daniela Herzog,Austria,female,1974-08-07,2019-02-14
4,Julia Jung,Austria,female,1991-01-04,2017-11-22
5,Juliane Kirsch,Austria,female,1977-03-01,2018-12-16


In [11]:
%%sql
SELECT *
FROM movies
LIMIT 5;

 * sqlite:///MovieNow.db
Done.


movie_id,title,genre,runtime,year_of_releas,renting_price
1,One Night at McCool's,Comedy,93,2001,2.09
2,Swordfish,Drama,99,2001,2.19
3,What Women Want,Comedy,127,2001,2.59
4,Training Day,Drama,122,2001,1.79
5,The Fellowship of the Ring,Science Fiction & Fantasy,178,2001,2.59


In [12]:
%%sql
SELECT *
FROM renting
LIMIT 5;

 * sqlite:///MovieNow.db
Done.


renting_id,customer_id,movie_id,rating,date_renting
1,41,8,None,2018-10-09
2,10,29,10.0,2017-03-01
3,108,45,4.0,2018-06-08
4,39,66,8.0,2018-10-22
5,104,15,7.0,2019-03-18


### Basic Information

#### What its the total number of the members?

In [20]:
%%sql
SELECT COUNT(DISTINCT name) AS number_of_members
FROM customers;

 * sqlite:///MovieNow.db
Done.


number_of_members
123


#### What is the total number of the movies?

In [38]:
%%sql
SELECT COUNT(DISTINCT title) AS number_of_movies
FROM movies;

 * sqlite:///MovieNow.db
Done.


number_of_movies
71


### Movie details

#### What is the top 5 movies that have the highest average rating?
Note: only choose movie that has more than 5 rating

In [58]:
%%sql
SELECT title,
       COUNT(rating) AS number_rating,
       ROUND(AVG(rating),2) AS avg_rating
FROM movies AS m
LEFT JOIN renting AS r
ON m.movie_id = r.movie_id
GROUP BY r.movie_id
HAVING number_rating > 5
ORDER BY avg_rating DESC
LIMIT 5;

 * sqlite:///MovieNow.db
Done.


title,number_rating,avg_rating
Django Unchained,6,9.33
Young Adult,6,8.83
Imagining Argentina,6,8.67
Harry Potter and the Half-Blood Prince,11,8.64
Morning Glory,7,8.43


#### What is the top 5 most rented movies?

In [59]:
%%sql
SELECT m.title, 
       COUNT(r.renting_id) AS number_rented
FROM renting AS r
LEFT JOIN movies AS m
ON r.movie_id = m.movie_id
GROUP BY r.movie_id
ORDER BY number_rented DESC
LIMIT 5;

 * sqlite:///MovieNow.db
Done.


title,number_rented
The Kingdom,15
Training Day,14
Harry Potter and the Half-Blood Prince,13
World Trade Center,13
Monster,13


#### How much income did each movie generate? 

In [70]:
%%sql
SELECT rm.title,
       ROUND(SUM(rm.renting_price),2) AS income_movie
FROM
       (SELECT m.title,  
               m.renting_price
       FROM renting AS r
       LEFT JOIN movies AS m
       ON r.movie_id=m.movie_id) AS rm
GROUP BY rm.title
ORDER BY income_movie DESC
LIMIT 5; 

 * sqlite:///MovieNow.db
Done.


title,income_movie
Bridget Jones - The Edge of Reason,37.57
Fair Game,34.68
The Kingdom,31.35
Two for the Money,30.69
Simone,29.59


### Customer details

#### 

#### Who are the customers who rent movies most often?

In [ ]:
%%sql
SELECT 